# Using matlab function as a metric in a tf.keras model

Text about background and why this can be useful.

## Imports and config parameters

In [1]:
import numpy as np
import pathlib
import tensorflow as tf
import matlab.engine

MATLAB_METRIC_DIR = 'matlab/'
IMAGE_H, IMAGE_W = 28, 28
IMAGE_CHANNELS = 1
N_CLASSES = 10

## Loading MNIST data

In [2]:
data = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = data.load_data()

print('train_set_images:', train_images.shape, type(train_images[0,0,0]), 
      'min:', np.min(train_images), 'max:', np.max(train_images))
print('train_set_labels:', train_labels.shape,  type(train_labels[0]), 
      'min:', np.min(train_labels), 'max:', np.max(train_labels))

def preprocess_mnist(images, labels):
    print('Preprocessing...')
    #One-hot-encoding of response variables 0-9
    labels = tf.keras.utils.to_categorical(labels)
    
    # Zero padding: (28,28) -> (32,32)
    #images = np.pad(images, [(0,0), (2,2), (2,2)], mode='constant', constant_values=0)
    
    # Rescaling: uint8(0-255) -> float32(0,1)
    images = tf.image.convert_image_dtype(images, tf.float32).numpy()
    return images, labels

train_images, train_labels = preprocess_mnist(train_images, train_labels)
test_images, test_labels = preprocess_mnist(test_images, test_labels)

print('train_set_images:', train_images.shape, type(train_images[0,0,0]), 
      'min:', np.min(train_images), 'max:', np.max(train_images))
print('train_set_labels:', train_labels.shape,  type(train_labels[0,0]), 
      'min:', np.min(train_labels), 'max:', np.max(train_labels))

train_set_images: (60000, 28, 28) <class 'numpy.uint8'> min: 0 max: 255
train_set_labels: (60000,) <class 'numpy.uint8'> min: 0 max: 9
Preprocessing...
Preprocessing...
train_set_images: (60000, 28, 28) <class 'numpy.float32'> min: 0.0 max: 1.0
train_set_labels: (60000, 10) <class 'numpy.float32'> min: 0.0 max: 1.0


## Starting matlab.engine

In [3]:
def start_matlab():
    print('Starting matlab.engine ...')
    eng = matlab.engine.start_matlab()
    eng.cd(str(pathlib.Path(MATLAB_METRIC_DIR).resolve()))
    if isinstance(eng, matlab.engine.matlabengine.MatlabEngine):
        print('matlab.engine started')
    return eng
matlab_engine = start_matlab()

Starting matlab.engine ...
matlab.engine started


## The matlab metrics functions

Wrapper functions that allow interaction with the tensorflow computational graph.

In [4]:
def py_matlab_accuracy(y_true, y_pred):
    y_true, y_pred = y_true.numpy(), y_pred.numpy()
    y_true, y_pred = np.transpose(y_true), np.transpose(y_pred)
    
    y_true, y_pred = matlab.double(y_true.tolist()), matlab.double(y_pred.tolist())
    metric = matlab_engine.accuracy(y_true, y_pred)
    return tf.constant(metric, tf.float32)

def matlab_accuracy(y_true, y_pred):
    matlab_metric = tf.py_function(py_matlab_accuracy, [y_true, y_pred], [tf.float32])
    return matlab_metric

## A simple convolutional model

In [5]:
def simple_conv_model():
    input_layer = tf.keras.Input(shape=(IMAGE_H, IMAGE_W, IMAGE_CHANNELS))
    x = tf.keras.layers.Conv2D(16, 5, activation="relu")(input_layer)
    x = tf.keras.layers.Conv2D(8, 5, activation="relu")(x)
    x = tf.keras.layers.Conv2D(4, 5, activation="relu")(x)
    x = tf.keras.layers.Conv2D(2, 5, activation="relu")(x)
    x = tf.keras.layers.Flatten()(x)
    output_layer = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(x)
    return input_layer, output_layer

input_layer, output_layer = simple_conv_model()
model = tf.keras.Model(input_layer, output_layer)

## Compiling our model with the matlab metric

In [6]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy', matlab_accuracy])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 16)        416       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 8)         3208      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 4)         804       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 2)         202       
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 10)               

## Fitting the model, watching the reported metrics

In [7]:
EPOCHS = 5
history = model.fit(x=train_images, y=train_labels, 
          batch_size=64, epochs = EPOCHS, steps_per_epoch = 200,
          validation_data=(test_images, test_labels))

Epoch 1/5
200/200 [==============================] - 11s 56ms/step - loss: 0.8180 - accuracy: 0.7402 - matlab_accuracy: 0.7402 - val_loss: 0.3571 - val_accuracy: 0.8915 - val_matlab_accuracy: 0.8908
Epoch 2/5
200/200 [==============================] - 11s 55ms/step - loss: 0.3166 - accuracy: 0.9064 - matlab_accuracy: 0.9064 - val_loss: 0.2187 - val_accuracy: 0.9343 - val_matlab_accuracy: 0.9340
Epoch 3/5
200/200 [==============================] - 11s 55ms/step - loss: 0.2325 - accuracy: 0.9302 - matlab_accuracy: 0.9302 - val_loss: 0.1729 - val_accuracy: 0.9477 - val_matlab_accuracy: 0.9474
Epoch 4/5
200/200 [==============================] - 11s 54ms/step - loss: 0.1831 - accuracy: 0.9452 - matlab_accuracy: 0.9452 - val_loss: 0.1374 - val_accuracy: 0.9579 - val_matlab_accuracy: 0.9581
Epoch 5/5
200/200 [==============================] - 11s 55ms/step - loss: 0.1581 - accuracy: 0.9533 - matlab_accuracy: 0.9534 - val_loss: 0.1369 - val_accuracy: 0.9573 - val_matlab_accuracy: 0.9575


## What if you want full control of the metric calculation?

Subclassing `tf.keras.Model` and customizing the `train_step()` and `test_step()` methods so that the matlab metric is only calculated when validating.

In [8]:
class CustomModel(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(CustomModel, self).__init__(*args, **kwargs)
        
        self.matlab_accuracy_fn = None
        self.matlab_accuracy_mean = None
        
    def compile(self, *args, **kwargs):
        super(CustomModel, self).compile(*args, **kwargs)
        self.matlab_accuracy_fn = matlab_accuracy
        self.matlab_accuracy_mean = tf.keras.metrics.Mean(name='matlab_accuracy')
        
    def train_step(self, data):
        super(CustomModel, self).train_step(data)
        
        metrics_to_report = {m.name: m.result() for m in self.metrics}
        metrics_to_report.pop(self.matlab_accuracy_mean.name)
        
        return metrics_to_report

    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        self.compiled_metrics.update_state(y, y_pred)
        
        matlab_accuracy_value = self.matlab_accuracy_fn(y, y_pred)
        self.matlab_accuracy_mean.update_state(matlab_accuracy_value)
        return {m.name: m.result() for m in self.metrics}
    
    @property
    def metrics(self):
        metrics = super().metrics
        metrics.append(self.matlab_accuracy_mean)
        return metrics

## Building and compiling our customized model

In [9]:
input_layer, output_layer = simple_conv_model()
custom_model = CustomModel(input_layer, output_layer)
custom_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

## Fitting the custom model, watching the reported metrics

In [10]:
EPOCHS = 5
history = custom_model.fit(x=train_images, y=train_labels, 
          batch_size=64, epochs = EPOCHS, steps_per_epoch = 100,
          validation_data=(test_images, test_labels))


Epoch 1/5
100/100 [==============================] - 5s 51ms/step - loss: 1.2353 - accuracy: 0.5861 - val_loss: 0.5508 - val_accuracy: 0.8310 - val_matlab_accuracy: 0.8315
Epoch 2/5
100/100 [==============================] - 5s 49ms/step - loss: 0.5142 - accuracy: 0.8475 - val_loss: 0.3978 - val_accuracy: 0.8835 - val_matlab_accuracy: 0.8838
Epoch 3/5
100/100 [==============================] - 5s 50ms/step - loss: 0.3845 - accuracy: 0.8886 - val_loss: 0.3379 - val_accuracy: 0.8996 - val_matlab_accuracy: 0.9001
Epoch 4/5
100/100 [==============================] - 5s 51ms/step - loss: 0.3239 - accuracy: 0.9055 - val_loss: 0.2611 - val_accuracy: 0.9218 - val_matlab_accuracy: 0.9219
Epoch 5/5
100/100 [==============================] - 5s 49ms/step - loss: 0.2518 - accuracy: 0.9258 - val_loss: 0.2162 - val_accuracy: 0.9381 - val_matlab_accuracy: 0.9381
